In [23]:
# Deps
import time 
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
import math
import pandas as pd
import warnings
import pickle
import GetSalary
import importlib
import scrape2_2 as scr2
import driver_config as dc
import numpy as np
import scipy.interpolate as si
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
importlib.reload(GetSalary)
importlib.reload(scr2)
importlib.reload(dc)

# Remove DevTools warning
#chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])


def fix_dict_lens(job_dict:dict) -> dict:

    ix = 0
    # Get dictionary lengths
    dic_lens = []
    for val in job_dict.values():
        dic_lens.append(len(val))
    
    # Making all dictionary entries lengths equal
    for i in range(len(dic_lens)):

        if dic_lens[i] == 6: # If Salary & Application Due are missing

            # The corresponding job in dicitonary
            job_dict['Job '+str(i+1)].insert(3, 'n/a') # Insert 'n/a' for Salary
            job_dict['Job '+str(i+1)].insert(5, 'n/a') # Insert 'n/a' for App. due
        
        if dic_lens[i] == 7: # If Salary or Application Due is missing

            if job_dict['Job '+str(i+1)][3].startswith('Salary'): # if salary is present
                # Insert 'n/a' for App. due
                job_dict['Job '+str(i+1)].insert(5, 'n/a')
            elif job_dict['Job '+str(i+1)][4].startswith('Application'): # else if App. Due is present (index 4 for length 7)
                # Insert 'n/a' for Salary
                job_dict['Job '+str(i+1)].insert(3, 'n/a')
            elif job_dict['Job '+str(i+1)][5].startswith('Announcement'): # In case of 'Annoucement' entry
                job_dict['Job '+str(i+1)].insert(3, 'n/a') # Insert 'n/a' for Salary
                job_dict['Job '+str(i+1)].insert(5, 'n/a') # Insert 'n/a' for App. due
                
                # get rid of 'Announcement' entry, which is 2 indexes more than before since we inserted n/a's
                del job_dict['Job '+str(i+1)][7]

        if dic_lens[i] == 9: # if all entries are full but we also have the 'Announcement', then simply remove that one
            if job_dict['Job '+str(i+1)][7].startswith('Announcement'):
                del job_dict['Job '+str(i+1)][7]

    return job_dict


In [2]:
with open('Job_Attrs_Programmer_Analyst'+'.pkl', 'rb') as f:
    job_attrs_Programmer_Analyst = pickle.load(f)

with open('Job_Attrs_IT_Support_and_Training'+'.pkl', 'rb') as f:
    job_attrs_IT_Support_and_Training = pickle.load(f)

with open('Job_Attrs_IT_Manager_Director'+'.pkl', 'rb') as f:
    job_attrs_IT_Manager_Director = pickle.load(f)

with open('Job_Attrs_workday'+'.pkl', 'rb') as f:
    job_attrs_workday = pickle.load(f)

len(job_attrs_Programmer_Analyst), len(job_attrs_IT_Support_and_Training), len(job_attrs_IT_Manager_Director), len(job_attrs_workday)

(151, 779, 339, 5082)

In [3]:
for i in range(5):
    print(len(job_attrs_Programmer_Analyst[i]),len(job_attrs_IT_Support_and_Training[i]), len(job_attrs_IT_Manager_Director[i]), len(job_attrs_workday[i]))

6 6 8 6
6 6 6 6
6 6 6 6
6 7 6 6
6 6 7 6


In [4]:
# Join all attrs together
all_attrs = job_attrs_Programmer_Analyst + job_attrs_IT_Support_and_Training + job_attrs_IT_Manager_Director + job_attrs_workday

print(len(all_attrs))

# Store in dictionary
job_dict = {}
for i in range(len(all_attrs)):
    job_dict['Job ' + str(i+1)] = all_attrs[i]

6351


In [7]:
for key, val in list(job_dict.items())[:10]:
    print(len(val))

job_dict = fix_dict_lens(job_dict)

print('\n')

for key, val in list(job_dict.items())[:10]:
    print(len(val))

6
6
6
6
6
6
7
6
6
6
7
6
6
6
6
6
6
7
6
7


8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8


In [13]:
# Check if a job does not meet the length criteria
for i in range(len(all_attrs)):
    if len(job_dict['Job ' + str(i+1)]) != 8:
        print(i)

927


In [21]:
print(len(job_dict['Job 928']))
job_dict['Job 928'].insert(3, 'n/a')
job_dict['Job 928'].insert(4, 'n/a')
job_dict['Job 928'].insert(5, 'n/a')
print(len(job_dict['Job 928']))

5
8


In [24]:
# April
df = pd.DataFrame(job_dict).T.rename(columns={0:'Title',1:'Location',2:'Type',3:'Salary',4:'Posted On',5:'Due',6:'Category',7:'Description'})
    
df.to_csv('April_Scrape' + '_Jobs.csv', index=False)

GetSalary.grab_salary_from_desc('April_Scrape_Jobs.csv') #

Final file saved as:  April_Scrape_Jobs_Updated.csv
